In [ ]:
%%capture
%pip install itables
%pip install openpyxl;
%pip install plotly;

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import yaml
from IPython.display import Markdown as md
import mercury as mr
from itables import show, JavascriptCode

pyo.init_notebook_mode()

# set all plotly background color to white:
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
file = mr.File(label=".xlsx file from Meitav Trade")
if file.filepath is not None:
    # verify the file name ends with .xlsx:
    if not file.filename.endswith('.xlsx'):
        raise ValueError('The file must be an .xlsx file')
    df = pd.read_excel(f'{file.filepath}')
else:
    df = pd.read_excel('demo_data.xlsx') # Change the file name to the name of the file you want to use by default

In [ ]:
# Filter only the required columns:
df = df[['שם נייר', 'מספר נייר', 'שווי נוכחי']] # (instrument_name, instrument_id, current_value in ILS ₪)

# Translate all the required columns to english:
df.columns = ['instrument_name', 'instrument_id', 'current_value']

In [ ]:
with open('instrument_details.yaml') as file:
    instrument_details = yaml.load(file, Loader=yaml.FullLoader)
    
# Add to the DF details about the instruments from instrument_details, if the instrument_name is not in the yaml file, add "unknown" to the details:
df['geo'] = df['instrument_name'].apply(lambda x: instrument_details['instruments'].get(x, {}).get('geo', 'unknown'))
df['type'] = df['instrument_name'].apply(lambda x: instrument_details['instruments'].get(x, {}).get('type', 'unknown'))
df['sub_type'] = df['instrument_name'].apply(lambda x: instrument_details['instruments'].get(x, {}).get('sub_type', 'unknown'))
df['asset_class'] = df['instrument_name'].apply(lambda x: instrument_details['instruments'].get(x, {}).get('asset_class', 'unknown'))

In [ ]:
md(f"# Total account value: {df['current_value'].sum():,.0f} ₪")

In [ ]:
# plot with a pie chart the current_value distribution of the instruments by type using plotly:
fig = px.pie(df, names='type', values='current_value', title='Current Value Distribution by Type')
fig.show()

In [ ]:
fig = px.pie(df, names='sub_type', values='current_value', title='Current Value Distribution by Sub Type')
fig.show()

In [ ]:
fig = px.pie(df, names='geo', values='current_value', title='Current Value Distribution by Geography')
fig.show()

In [ ]:
fig = px.pie(df, names='asset_class', values='current_value', title='Current Value Distribution by Asset Class')
fig.show()

# Account Holdings:

In [ ]:
# Create printable_df where the current_value is formatted to be in ₪ with 0 decimal points and with thousands separator:
printable_df = df.copy()
# Set the instrument_id as the last column:
printable_df = printable_df[
    ['instrument_name', 'current_value', 'geo', 'type', 'sub_type', 'asset_class', 'instrument_id']]

# print all rows in printable_df using itables:
show(printable_df, style='display nowrap', allign='left', width='100%', paging=False,
     columnDefs=[{"className": "dt-left", "targets": "_all"},
                 {"width": "25%", "targets": [0]},
                 {"render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 0, '₪')"), "targets": [1]}
                 ]
     )